In [1]:
#Importing Packages I'll need
import pandas as pd
import numpy as np

# Initial Loading

In [2]:
#Loading the main Opportunity Atlas file. This will only work if the file below is in the same folder as this notebook
OA = pd.read_csv("tract_outcomes_early.csv")
OA.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5703) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,state,county,tract,kir_natam_female_p1,kir_natam_female_p25,kir_natam_female_p50,kir_natam_female_p75,kir_natam_female_p100,kir_natam_female_n,kir_natam_female_mean,...,jail_white_pooled_mean_se,kfr_white_pooled_mean_se,staytract_white_pooled_mean_se,staycz_white_pooled_mean_se,stayhome_white_pooled_mean_se,working_white_pooled_mean_se,kir_stycz_white_pooled_mean_se,kfr_stycz_white_pooled_mean_se,spouse_rk_white_pooled_mean_se,par_rank_white_pooled_mean_se
0,1,1,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007490,0.014829,0.020353,0.025752,0.019710,0.025348,0.018290,0.017890,0.018882,0.013778
1,1,1,20200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003828,0.020797,0.028821,0.041338,0.028252,0.029613,0.030380,0.028010,0.025328,0.021874
2,1,1,20300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002141,0.011726,0.016670,0.022392,0.015206,0.019161,0.015177,0.015285,0.016145,0.009353
3,1,1,20400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001777,0.010787,0.016027,0.019096,0.012477,0.016844,0.014253,0.014508,0.014396,0.009628
4,1,1,20500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000983,0.008789,0.012861,0.017491,0.010930,0.014396,0.011836,0.012748,0.010329,0.006667


In [3]:
#Loading the newest Business Master File Data. Past data tbd.
BMF = pd.read_csv("eo1.csv")
BMF.head()

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,...,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
0,19818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,3514
1,29215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,...,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN
2,587764,IGLESIA BETHESDA INC,NaN,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X21,NaN
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,NaN,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X21,NaN
4,765634,MERCY CHAPEL INTERNATIONAL,NaN,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,...,0,0,6,0,12,NaN,NaN,NaN,X20,NaN


In [ ]:
#The Business Master File is more granular than Opportunity Atlas, so we have to match BMF to the minimum shared
#granularity: 
